In [0]:
import cv2
import os
import numpy as np
import matplotlib as plt
from glob import glob
import zipfile
from datetime import datetime

%matplotlib inline

In [23]:
#  montar google driver
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Importar Dados e tratar os dados

In [0]:
# extrai do Zip os csv's
fzp = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/dataset.zip')
fzp.extractall('.') 
fzp.close()

In [0]:
# dog = 0
# cat = 1
class_name = ['dog','cat']

In [0]:
dir_img_train = 'dataset/train/*.jpg'
dir_img_test = 'dataset/test/*.jpg'

In [0]:
def convert_img_array(lst):
    """
    lista diretorios com imagens e convertem array
    
    """
    lst_img, lst_label=[],[]
    for d in glob(lst):
        lst_img.append(cv2.imread(d))
        if 'dog' in d:
            lst_label.append('0')
        elif 'cat' in d:
            lst_label.append('1')
    return np.array(lst_img), np.array(lst_label)

In [0]:
# gerando dados de trainos e test
train_img, train_label = convert_img_array(dir_img_train)
test_img, test_label = convert_img_array(dir_img_test)

In [30]:
train_img.shape

(8000, 32, 32, 3)

In [31]:
test_label.shape

(2000,)

In [32]:
train_img.shape

(8000, 32, 32, 3)

In [33]:
train_label.shape

(8000,)

In [0]:
# escalar as imagens para que o valor esteja entre 0 e 1
train_img = train_img / 255.0
test_img = test_img / 255.0

### Classificação com Keras

Melhor acurácia utilizando optimizer('sgd') = 0.7225

In [0]:
!pip install -U --ignore-installed tensorboardcolab

  Created wheel for tensorboardcolab: filename=tensorboardcolab-0.0.22-cp36-none-any.whl size=3860 sha256=946c93a5609453a9386ff861b46a0456289d6c781d7739a84b5153684fd81acb
  Stored in directory: /root/.cache/pip/wheels/c4/aa/a0/3aaf4f1a66adbdab9b7bdd4c96db8ada89eb7cd87200cfdd32
Successfully built tensorboardcolab


In [0]:
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow.keras import losses
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping


In [0]:
# Estaciar Modelo
model = Sequential()

# Camada de Convolução
model.add(Conv2D(32,(5,5),padding='same', input_shape = (32, 32, 3), activation=tf.nn.relu))
model.add(Conv2D(64,(5,5),padding='same', activation=tf.nn.relu))

# Camada de Polling
model.add(MaxPooling2D(pool_size=(2,2)))

# camada de Dropout
model.add(Dropout(0,25))

# Flattening
model.add(Flatten())

# Full conection
model.add(Dense(256, activation = tf.nn.relu))
model.add(Dense(256, activation = tf.nn.relu))

# camada de Dropout
model.add(Dropout(0.5))

model.add(Dense(2, activation = tf.nn.softmax))

In [0]:
# compilar modelo
model.compile(optimizer='adam', 
              loss=losses.sparse_categorical_crossentropy,
              metrics=['accuracy']
             )

In [0]:
# treinar modelo
start = datetime.now()

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)


treino = model.fit(train_img, 
          train_label,
          batch_size = 32,
          epochs = 1000,
          validation_split=0.2,
          verbose = 1,
          callbacks=[learning_rate_reduction])



print('')
print('Tempo decorrido para treinar o modelo: ', datetime.now() - start)

Train on 6400 samples, validate on 1600 samples
Epoch 1/1000


AttributeError: ignored

In [0]:
# Eficiência do modelo
test_loss, test_acc = model.evaluate(test_img, test_label)
print('Acurácia:', test_acc)
print('Perda:', test_loss)

2000/2000 [==============================] - 0s 103us/sample - loss: 6.9231 - acc: 0.5460
Acurácia: 0.546
Perda: 6.923054647445679


In [0]:
# previsoes
predict = model.predict(test_img)

In [0]:
# previsão é um vetor de 2 números onde descreve a confiança que teve para cada resultado
p = 600

print('Predito:', class_name[np.argmax(predict[p])])
print('Real:', class_name[int(test_label[p])])

Predito: cat
Real: cat


### Classificação com Machine Learning





In [0]:
from sklearn.metrics import accuracy_score

In [0]:
# padroniza a entrada com esse reshape
# convert de 3d para 2d
def _reshape(ds):
    return ds.reshape(len(ds),-1)

In [0]:
x_train = _reshape(train_img)
y_test = _reshape(test_img)

#### Modelo SVM 

Melhor acurácia = 0.62

In [0]:
from sklearn.svm import SVC

In [0]:
# accuracy testada do modelo
# 'linear' =  0.5935
# 'poly' = 0.566
# 'rbf' = 0.6195
# 'sigmoid' = 0.5885

model_svm = SVC(kernel='rbf')
model_svm.fit(x_train, train_label)

predicted = model_svm.predict(y_test)
accuracy_score(test_label, predicted)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.6195

#### Regressão Logistica

Melhor acurácia = 0.597 com 'sag'

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
model_log = LogisticRegression(solver='sag')

In [0]:
model_log.fit(x_train, train_label)
pred_log1 = model_log.predict(y_test)
accuracy_score(test_label, pred_log1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.597

#### Floresta aleatória (Random Florest)

Melhor acurácia = 0.575

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
model_rd = RandomForestClassifier()

In [0]:
model_rd.fit(x_train, train_label)
pred_rd = model_rd.predict(y_test)
accuracy_score(test_label, pred_rd)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.575

#### Naives Bayes

Melhor acurácia = 0.575

In [0]:
from sklearn.naive_bayes import GaussianNB

In [0]:
model_nb = GaussianNB()

In [0]:
model_nb.fit(x_train, train_label)
pred_nb = model_rd.predict(y_test)
accuracy_score(test_label, pred_nb)

0.575

#### K-Nearest Neighbors (KNN)

Melhor acurácia = 0.582

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
model_knn = KNeighborsClassifier()

In [0]:
model_knn.fit(x_train, train_label)
pred_knn = model_knn.predict(y_test)
accuracy_score(test_label, pred_knn)

0.5825